In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'E:/Dissertation Data'
datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\tom\\Google Drive\\University\\Dissertation\\Code'

We collect lexical co-occurrence statistics on all words in
the English Wikipedia, using the WikiExtractor tool2 to retrieve
plain text from the April 2015 dump (ca. 2.8B words),
and using simple regular expressions to segment sentences
and words, and remove URLs and punctuation. We perform
no POS tagging, lemmatisation, case normalisation,
or removal of numbers or symbols.

In [3]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [4]:
w10p = PlaintextCorpusReader(datapath+'/Corpora/wiki/enwiki_20200520/','enwiki_20200520_10pc.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [5]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

We collect word frequency information with the
SRILM language modelling toolkit (Stolcke, 2002), counting
n-grams (n <= 3), treating MWEs as contiguous bigrams
and trigrams), and identify MWE candidates by computing
the Poisson collocation measure (Quasthoff and Wolff,
2002) for all bigrams and trigrams (ca. 23M n-grams).
This method should be readily extensible to include longer
n-grams.

In [4]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [7]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/enwiki_20200520/10pc_stop.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [8]:
stop

{'0',
 '1',
 '2',
 '3',
 'A',
 'He',
 'In',
 'It',
 'New',
 'The',
 'a',
 'also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'been',
 'but',
 'by',
 'first',
 'for',
 'from',
 'had',
 'has',
 'have',
 'he',
 'her',
 'his',
 'in',
 'is',
 'it',
 'its',
 'not',
 'of',
 'on',
 'one',
 'or',
 'that',
 'the',
 'their',
 'this',
 'to',
 'two',
 'was',
 'were',
 'which',
 'who',
 'with'}

In [9]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/10pc_ngram_eval.pkl')

ngram_eval

,ngram,poisson,len,batch
0,"(of, the)",3.874652e+06,2,-2
1,"(References, External, links)",2.566994e+06,3,1
2,"(External, links)",2.229096e+06,2,2
3,"(in, the)",2.094387e+06,2,-2
4,"(0, 0, 0)",1.798530e+06,3,-2
...,...,...,...,...
499995,"(Work, started, on)",3.883693e+02,3,8
499996,"(of, these, techniques)",3.883679e+02,3,8
499997,"(Stadio, Flaminio)",3.883678e+02,2,1
499998,"(IRE, 3, C)",3.883678e+02,3,1


In [10]:
batch_count = max(ngram_eval.batch)

In [11]:
ngram_eval.batch.value_counts()

 1     94011
 2     79538
 3     69221
 4     56154
 5     48934
 6     38288
 7     30374
-1     29076
 8     22495
 9     19272
 10    11300
-2      1337
Name: batch, dtype: int64

In [12]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [18]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [19]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/wiki10pc_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 10
Loading w2v model
Gathering MWE stats
 MWE 0/94011: References+External+links
 MWE 5000/94011: 2016–17+season
 MWE 10000/94011: said+she
 MWE 15000/94011: Nuevo+Laredo
 MWE 20000/94011: Campeonato+de+Portugal
 MWE 25000/94011: going+down
 MWE 30000/94011: was+convicted+in
 MWE 35000/94011: begin+work+on
 MWE 40000/94011: Julien+Duvivier
 MWE 45000/94011: municipal+unit+has
 MWE 50000/94011: Bobby+Hutcherson
 MWE 55000/94011: Little+Feat
 MWE 60000/94011: Multiple+Launch+Rocket
 MWE 65000/94011: Institutional+Investor
 MWE 70000/94011: Guillermo+Coria
 MWE 75000/94011: Delta+Connection
 MWE 80000/94011: reduce+energy+consumption
 MWE 85000/94011: consist+primarily+of
 MWE 90000/94011: would+entail
Processing batch 2 of 10
Loading w2v model
Gathering MWE stats
 MWE 0/79538: External+links
 MWE 5000/79538: Liberal+Democratic+Party
 MWE 10000/79538: forced+out+of
 MWE 15000/79538: electricity+generation
 MWE 20000/79538: the+wreck
 MWE 25000/79538: and+Merrie+Melod

In [20]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Results/wiki10pc_output_001.csv', index=False)

Appending batch 1 of 10
Appending batch 2 of 10
Appending batch 3 of 10
Appending batch 4 of 10
Appending batch 5 of 10
Appending batch 6 of 10
Appending batch 7 of 10
Appending batch 8 of 10
Appending batch 9 of 10
Appending batch 10 of 10


In [21]:
all_batches

,ngram,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(1980s, 1970s, 1960s)",504.762099,3,8,"[0, 0, 0]","[0.13252905, 0.2195085, 0.16913609, 0.18139634...","[[0.8419779, -0.8224145, -1.1535275, -0.101147...","[-0.23996054, -0.23782371, -0.2340202]",-0.237268
1,"(amongst, many, others)",712.565055,3,1,"[0, 0, 0]","[0.014603417, 0.11368454, 0.1424945, 0.1589423...","[[1.1393661, -0.18046533, -0.26049742, 0.29468...","[-0.15940933, -0.21269974, -0.19618905]",-0.189433
2,"(European, register, of)",588.560764,3,1,"[0, 0, 1]","[0.14991252, 0.15334864, 0.24452145, 0.3467293...","[[0.088441856, -1.0335438, 0.83159375, -2.4368...","[-0.1621425, -0.21278596, nan]",-0.187464
3,"(1950s, 1940s, 1930s)",576.491489,3,10,"[0, 0, 0]","[0.20758897, 0.26073128, 0.2666307, 0.3133268,...","[[-0.43351597, -0.032804497, -0.5531056, -1.58...","[-0.18906975, -0.1606813, -0.19921347]",-0.182988
4,"(As, far, back)",449.645748,3,3,"[0, 0, 0]","[-0.009685142, 0.012536515, 0.1396035, 0.10224...","[[-0.2582686, 0.022926275, -0.18216193, -0.799...","[-0.09595686, -0.2087354, -0.24125503]",-0.181982
...,...,...,...,...,...,...,...,...,...
499995,"(Billboard, R_B, Albums)",482.775787,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499996,"(eligible, Not, eligible)",459.896112,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499997,"(Change, Seats, Change)",450.553474,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499998,"(A_M, University, System)",425.385206,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
